## Preliminary

This notebook explains you how to build your 1st plugin. The plugin is a Python script uploaded which will be triggered based on an event you define. For instance you can trigger a plugin each time a labeler click on submit with the `on_submit` handler
The plugin should have separate methods for the different types of events : at the moment these are *on_submit* and *on_review*. These methods will have a predefined set of parameters


Therefore, the skeleton of the plugin should be :

```python
from kili.plugins import PluginCore
from kili.types import Label

class Plugin(PluginCore):
    """
    Custom plugin instance
    """

    def handle_label(self):
        #custom methods 
        print('Custom method handle label')

    def on_review(self, label: Label, project_id: str, asset_id: str) -> None:
        """
        Dedicated handler for Review action 
        """
        return super().on_review(label, project_id, asset_id)

    def on_submit(self, label: Label, project_id: str, asset_id: str) -> None:
        """
        Dedicated handler for Review action 
        """
        # Use kili for actions with 
        print(self.kili.count_projects(project_id=project_id))
        print('On submit called')
        self.handle_label()

```

**IMPORTANT** : Imports of packages inside the plugin are not permitted at the moment (except for `numpy`).

## Instantiate Kili with your personal API_KEY

In [1]:
#%pip install kili

from kili.client import Kili
import os

api_endpoint = os.getenv('KILI_API_ENDPOINT')
# If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'
api_endpoint = 'https://staging.cloud.kili-technology.com/api/label/v2/graphql'

api_key = os.getenv('KILI_ADMIN_API_KEY_STAGING')

kili = Kili(api_endpoint=api_endpoint, api_key=api_key)

/Users/philippedesaintchamas/opt/anaconda3/envs/plugins/lib/python3.8/site-packages/kili/authentication.py:103: UserWarning: Kili Python SDK version should match with Kili API version.
Please install version: "pip install kili==2.125.0"
  warnings.warn(message, UserWarning)


## Develop your future plugin

The first step is to define the functions that will be called when the event is triggered. You will be able to iterate on these functions locally with the help of the next section. 

This cell should be the content of the `.py` file that you will upload as a plugin at the end.

In [2]:
from kili.plugins import PluginCore
from kili.types import Label

class Plugin(PluginCore):
    """
    Custom plugin instance
    """

    def handle_label(self):
        #custom methods 
        print('Custom method handle label')

    def on_review(self, label: Label, project_id: str, asset_id: str) -> None:
        """
        Dedicated handler for Review action 
        """
        return super().on_review(label, project_id, asset_id)

    def on_submit(self, label: Label, project_id: str, asset_id: str) -> None:
        """
        Dedicated handler for Review action 
        """
        # Use kili for actions with 
        print(self.kili.count_projects(project_id=project_id))
        print('On submit called')
        self.handle_label()

### Testing locally the plugin

This section will allow you to test your plugin locally before uploading it.

In [3]:
import traceback

# instantiate the plugin
my_plugin_instance = Plugin(kili)

def runPlugin(label, project_id, asset_id, call_on_submit = True, call_on_review = True):
    """
    This function is a wrapper around the actual onSubmit and onReview functions.
    In case there is an error, it will catch it and print its name. The trace of the
    error will not be saved in the actual logs of the plugin that you will be able to
    retrieve after uploading the plugin, but here they will be printed to help you
    in the development of the plugin.

    If you want to have the full trace of the error, you can call directly the functions
    onSubmit and onReview without using this wrapper, for example :
    >>> onSubmit(projectId=project_id,assetId=asset_id,label=label)

    You can also pass the values of call_on_submit and call_on_review to test
    only certain events.
    """
    try:
        if call_on_submit:
            my_plugin_instance.on_submit(project_id=project_id,asset_id=asset_id,label=label)

        if call_on_review:
            my_plugin_instance.on_review(project_id=project_id,asset_id=asset_id,label=label)
    except Exception as e:
        print(repr(e))
        traceback.print_exc() # This will not be saved in the logs of the actual plugin,
                              # but it can help when iterating over the development of the plugin

def get_label(label_id, project_id):
    """
    Function to get the object Label with the same keys as it will be in the plugin
    """
    label = kili.labels(
            project_id=project_id,
            label_id=label_id,
            fields=['id', 'jsonResponse', 'author.id', 'labelType', 'createdAt', 'secondsToLabel']
        )[0]

    label['authorId'] = label['author']['id']
    del label['author']
    return label

### Test the plugin run

If you already have created a project for testing and there are some labels created, you can use directly their ids and use the following cell. Otherwise, you can follow the notebook *plugins_example.ipynb* to create a new project, upload an asset and an associated label.

In [4]:
project_id = ''
asset_id = ''
label_id = ''

label = get_label(label_id=label_id, project_id=project_id)

runPlugin(project_id=project_id, asset_id=asset_id,label=label)

  0%|          | 100/159150 [00:04<1:55:50, 22.88it/s]


KeyboardInterrupt: 

In [ ]:
path_to_plugin = 'path/to/file/plugin.py'
plugin_name = 'plugin name'
project_id = 'project_id'

kili.upload_plugin_beta(path_to_plugin, plugin_name)

kili.activate_plugin_on_project(plugin_name, project_id=project_id)

## Monitoring the plugin
Important to know is that the creation of the plugin takes some time (around 5 minutes), so after that time the plugin will begin to be run (if labeling events will be triggered on this project) and you will be able to get the logs of the runs. To do that, you can use:

In [ ]:
import json
from datetime import date
from datetime import datetime
dt = date.today()  # You can change this date if needed
start_date = datetime.combine(dt, datetime.min.time())

logs = kili.get_plugin_logs(project_id=project_id, plugin_name=plugin_name, start_date=start_date)

logs_json = json.loads(logs)
print(json.dumps(logs_json, indent=4))

## Managing your plugin
You also have several other methods to manage your plugins and their lifecycle.

In [3]:
# Get the list of all uploaded plugins in your organization
plugins = kili.list_plugins()

In [4]:
# Get the status of a plugin
kili.get_plugin_status(plugin_name='plugin')

GraphQLError: error: "[pluginsError] Error from label-plugin-runner -- This can be due to: 404: Not Found: createPluginRunner: Plugin [name: plugin] | trace : Error: 404: Not Found: createPluginRunner: Plugin [name: plugin]
    at prettifyPluginsManagerError (/snapshot/app/dist/resolvers/pluginsManager/helpers.js)
    at getPluginRunnerStatus (/snapshot/app/dist/resolvers/pluginsManager/queries.js)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)
    at async /snapshot/app/dist/services/error/index.js"

In [ ]:
# Update a plugin with new source code
updated_path = 'path/to/updated/file.py'
kili.update_plugin(plugin_name=plugin_name, file_path=updated_path)

In [ ]:
# Deactivate the plugin on a certain project (the plugin can still be active for other projects)
kili.deactivate_plugin_on_project(plugin_name=plugin_name, project_id=project_id)

In [ ]:
# Delete the plugin completely (deactivates automatically the plugin from all projects)
kili.delete_plugin(plugin_name=plugin_name)

In [8]:
plugins

[{'name': 'Load_test_plugin3',
  'id': 'clagwpboh0001019jbbow4tw7',
  'createdAt': '2022-11-14T14:54:17.441Z',
  'updatedAt': '2022-11-14T14:55:49.455Z'},
 {'name': 'my_new_plugin_',
  'id': 'clagyxq080001017j5k934yyb',
  'createdAt': '2022-11-14T15:56:48.488Z',
  'updatedAt': '2022-11-14T15:58:35.730Z'},
 {'name': 'plugin for list',
  'id': 'clai4r585000501e750fd4id8',
  'createdAt': '2022-11-15T11:27:25.493Z',
  'updatedAt': '2022-11-15T11:29:12.635Z'},
 {'name': 'toto',
  'id': 'clai0brir001d01811mxbfa6y',
  'createdAt': '2022-11-15T09:23:29.427Z',
  'updatedAt': '2022-11-15T09:25:02.313Z'}]